# Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import json

### Load the data into Pandas dataframe

In [ ]:
def clean_dataframe(dpath, dpath_hard):
    
    with open(dpath, 'r') as file:
        data = json.load(file)

    flattened_data = []
    for key, value in data.items():
        row = {'id_EXIST': key}
        for k, v in value.items():
            row[k] = v
        flattened_data.append(row)

    df = pd.DataFrame(flattened_data)
    df = df [['id_EXIST', 'tweet']]
    df.rename(columns={"id_EXIST": "id", "tweet": "text"}, inplace = True)
    
    
    with open(dpath_hard, 'r') as file:
        data = json.load(file)    

    df_hard = pd.DataFrame(data)
    df_hard = df_hard[['id', 'value']]
    df_hard.rename(columns={'value': 'label'}, inplace=True)
    
    df_comb = pd.merge(df, df_hard, on = 'id', how = 'inner')
    
    return df_comb
    
    

In [ ]:
dpath_train = "/kaggle/input/exist-task-2/EXIST2024_training.json"
dpath_train_hard = "/kaggle/input/exist-task-2/EXIST2024_training_task2_gold_hard.json"

# df_hard.head()

df_train = clean_dataframe(dpath_train, dpath_train_hard)
df_train.head()

Drop the columns that are not required for the neural network.

In [ ]:
dpath_test = "/kaggle/input/exist-task-2/EXIST2024_dev.json"
dpath_test_hard = "/kaggle/input/exist-task-2/EXIST2024_dev_task2_gold_hard.json"

# df_hard.head()

df_test = clean_dataframe(dpath_test, dpath_test_hard)
df_test.head()

Understand the distribution better.

In [ ]:
sns.countplot(df_train.label)
plt.xlabel('label')
plt.title('Number of sexist and not_sexist messages')

* Create input and output vectors.
* Process the labels.

In [ ]:
sns.countplot(df_test.label)
plt.xlabel('label')
plt.title('Number of sexist and not_sexist messages')

In [ ]:
# X = df.v2
# Y = df.v1
# le = LabelEncoder()
# Y = le.fit_transform(Y)
# Y = Y.reshape(-1,1)

In [ ]:
# Y_train

In [ ]:
X_train = df_train.text
Y_train = df_train.label
encoder = OneHotEncoder(sparse=False)
Y_train = encoder.fit_transform(Y_train.values.reshape(-1, 1))

In [ ]:
Y_train

In [ ]:
X_test = df_test.text
Y_test = df_test.label
encoder = OneHotEncoder(sparse=False)
Y_test = encoder.fit_transform(Y_test.values.reshape(-1, 1))

Split into training and test data.

In [ ]:
# X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [ ]:
max_words = 1000
max_len = 80
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
# Y_train

### RNN
Define the RNN structure.

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4, name='out_layer')(layer)  # Update number of units to match num_classes
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

Call the function and compile the model.

In [ ]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Fit on the training data.

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=20,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.8f}\n  Accuracy: {:0.8f}'.format(accr[0],accr[1]))

In [ ]:
test_predictions = model.predict(test_sequences_matrix)

In [ ]:
test_predictions

In [ ]:
# Find the index of the maximum value in each row
max_indices = np.argmax(test_predictions, axis=1)

# Create a new array of zeros with the same shape as the original array
result = np.zeros_like(test_predictions)

# Set the maximum value positions to 1
result[np.arange(len(test_predictions)), max_indices] = 1


result

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
print(classification_report(result, Y_test, digits = 4))

source for attention: [https://github.com/philipperemy/keras-attention-mechanism/blob/master/attention_lstm.py](http://)
source for other code: [https://www.kaggle.com/kredy10/simple-lstm-for-text-classification/notebook](http://)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
SINGLE_ATTENTION_VECTOR = False
APPLY_ATTENTION_BEFORE_LSTM = False
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    # output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [ ]:
def model_attention_applied_after_lstm():
    #inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    
    lstm_units = 64
    lstm_out = LSTM(lstm_units, return_sequences=True)(layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(4, activation='softmax')(attention_mul)  # Update for multi-class
    model = Model(input=[inputs], output=output)
    return model

In [ ]:
from keras.layers import merge
from keras.layers import multiply
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

from keras.utils.vis_utils import plot_model

In [ ]:
INPUT_DIM = 50
TIME_STEPS = max_len

In [ ]:
sequences_matrix.shape,Y_train.shape

In [ ]:
m = model_attention_applied_after_lstm()

In [ ]:
m.summary()

In [ ]:
m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
m.fit(sequences_matrix,Y_train,batch_size=128,epochs=20,
          validation_split=0.2)

In [ ]:
accr = m.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.8f}\n  Accuracy: {:0.8f}'.format(accr[0],accr[1]))

In [ ]:
test_predictions = m.predict(test_sequences_matrix)

In [ ]:
test_predictions

In [ ]:
Y_test

In [ ]:
# Find the index of the maximum value in each row
max_indices = np.argmax(test_predictions, axis=1)

# Create a new array of zeros with the same shape as the original array
result = np.zeros_like(test_predictions)

# Set the maximum value positions to 1
result[np.arange(len(test_predictions)), max_indices] = 1

result

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
print(classification_report(result, Y_test, digits = 4))